<a href="https://colab.research.google.com/github/nisarahamedk/kaggle-riid/blob/master/notebooks/riid_tfrecords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install gcsfs

In [2]:

import math

import gcsfs
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
import tensorflow as tf

In [3]:
# DATA_PATH = "/kaggle/input/riiid-test-answer-prediction/"
DATA_PATH = "gs://kds-e80dfc3d272252bbf34c627d756f891826dab0c19f30ec0fc3ac1979"

In [4]:

dtypes_train = {
#     'row_id': 'int64',
#     'timestamp': 'int64',
    'user_id': 'int32',
    'content_id': 'int16',
#     'content_type_id': 'int8',
    'task_container_id': 'int16',
#     'user_answer': 'int8',
    'answered_correctly': 'int8',
    'prior_question_elapsed_time': 'float32',
#     'prior_question_had_explanation': 'boolean'
    }

dtypes_questions = {
    "question_id": "",
    "bundle_id": "",
    "correct_answer": "",
    "part": "",
    "tags": "",
}

dtypes_lectures = {
    "lecture_id": "",
    "part": "",
    "tag": "",
    "type_of": "",
}

In [5]:
train_df = pd.read_csv(DATA_PATH + "/train.csv", dtype=dtypes_train, usecols=dtypes_train.keys(), nrows=1e6)
train_df['prior_question_elapsed_time'] = train_df["prior_question_elapsed_time"].fillna(0).astype(np.float32) / 60000
train_df.head()

,user_id,content_id,task_container_id,answered_correctly,prior_question_elapsed_time
0,115,5692,1,1,0.000000
1,115,5716,2,1,0.616667
2,115,128,0,1,0.916667
3,115,7860,3,1,0.316667
4,115,7922,4,1,0.183333


In [6]:
user_groups = train_df.groupby("user_id")
user_groups_arr = user_groups.apply(
    lambda rows: (
        rows["content_id"].values, 
        rows["task_container_id"].values, 
        rows["prior_question_elapsed_time"].values,
        # rows["part"].values,
        rows["answered_correctly"].values,
        )
    )
user_groups_arr[115][0]

array([5692, 5716,  128, 7860, 7922,  156,   51,   50, 7896, 7863,  152,
        104,  108, 7900, 7901, 7971,   25,  183, 7926, 7927,    4, 7984,
         45,  185,   55, 7876,    6,  172, 7898,  175,  100, 7859,   57,
       7948,  151,  167, 7897, 7882, 7962, 1278, 2065, 2064, 2063, 3363,
       3365, 3364], dtype=int16)

In [7]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=value)) # changed [value] to val

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=value)) # changed [value] to value

In [18]:
def serialize_example(user_arr):
  feature = {
      "content_id": _bytes_feature(tf.io.serialize_tensor(user_arr[0])),
      "task_container_id": _bytes_feature(tf.io.serialize_tensor(user_arr[1])),
      "prior_question_elapsed_time": _bytes_feature(tf.io.serialize_tensor(user_arr[2])),
      "answered_correctly": _bytes_feature(tf.io.serialize_tensor(user_arr[3])),
  }
  proto_example = tf.train.Example(features=tf.train.Features(feature=feature))
  serialized_example = proto_example.SerializeToString()
  return serialized_example

In [75]:
feature_desc = {
    "content_id": tf.io.FixedLenFeature([], tf.string),
    "task_container_id": tf.io.FixedLenFeature([], tf.string),
    "prior_question_elapsed_time": tf.io.FixedLenFeature([], tf.string),
    "answered_correctly": tf.io.FixedLenFeature([], tf.string),
}

In [76]:
example = serialize_example(user_groups_arr[115])
tf.io.parse_tensor(tf.io.parse_single_example(example, feature_desc)["content_id"], tf.int16)

<tf.Tensor: shape=(46,), dtype=int16, numpy=
array([5692, 5716,  128, 7860, 7922,  156,   51,   50, 7896, 7863,  152,
        104,  108, 7900, 7901, 7971,   25,  183, 7926, 7927,    4, 7984,
         45,  185,   55, 7876,    6,  172, 7898,  175,  100, 7859,   57,
       7948,  151,  167, 7897, 7882, 7962, 1278, 2065, 2064, 2063, 3363,
       3365, 3364], dtype=int16)>

In [77]:
def create_tfrecords_from_user_groups(user_groups_arr):

  with tf.io.TFRecordWriter("tfrecoreds.tfrec") as writer:
    for user_arr in user_groups_arr:
      example = serialize_example(user_arr)

      writer.write(example)

In [78]:
create_tfrecords_from_user_groups(user_groups_arr)

In [79]:
def parse_example(example):
  example = tf.io.parse_single_example(example, feature_desc)

  content_id = tf.io.parse_tensor(example["content_id"], tf.int16)
  task_container_id = tf.io.parse_tensor(example["task_container_id"], tf.int16)
  prior_question_elapsed_time = tf.io.parse_tensor(example["prior_question_elapsed_time"], tf.float32)
  answered_correctly = tf.io.parse_tensor(example["answered_correctly"], tf.int8)
  
  return (
      content_id,
      task_container_id,
      prior_question_elapsed_time,
      answered_correctly
  )

In [80]:
filenames = tf.io.gfile.glob("*.tfrec")
filenames

['./tfrecoreds.tfrec']

In [81]:
dataset = tf.data.TFRecordDataset(filenames)

In [82]:
dataset = dataset.map(parse_example)

In [83]:
b = 0
for item in dataset:
  b += 1
b

3824